# Prompt Engineering Analysis with Anthropic's Claude

This notebook analyzes how different prompt engineering techniques affect Claude's output distributions, response characteristics, and behavior.

## Topics Covered:
1. Setting up the Anthropic API
2. Comparing different prompt engineering techniques
3. Analyzing temperature effects on output distribution
4. Visualizing output diversity and characteristics
5. Statistical analysis of prompt effectiveness

## Setup

In [ ]:
with open(".key", 'r') as fp:
    API_KEY = fp.read()
MODEL_NAME = "claude-3-haiku-20240307"

# Stores the API_KEY & MODEL_NAME variables for use across notebooks within the IPython store
%store API_KEY
%store MODEL_NAME


In [ ]:
import anthropic
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any
from collections import Counter
import time

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Import our analyzer
from prompt_engineering_analyzer import PromptEngineeringAnalyzer

print("Setup complete!")

## Configure API Key

Make sure you have your Anthropic API key set as an environment variable:
```bash
export ANTHROPIC_API_KEY='your-api-key-here'
```

Or set it directly in this notebook (not recommended for production):

In [ ]:
# Option 1: Use environment variable (recommended)
# Option 2: Set directly (uncomment and use with caution)
api_key = API_KEY

if not API_KEY:
    print("⚠️ WARNING: ANTHROPIC_API_KEY not found!")
    print("Please set your API key before proceeding.")
else:
    print("✓ API key configured")

## Initialize Analyzer

In [ ]:
# Initialize the analyzer with your preferred model
analyzer = PromptEngineeringAnalyzer(
    api_key=api_key,
    model=MODEL_NAME # or "claude-3-opus-20240229", etc.
)

print(f"Analyzer initialized with model: {analyzer.model}")

## 1. Basic Prompt Testing

Let's start with a simple example to see how Claude responds to a basic prompt.

In [ ]:
# Test a simple prompt
test_prompt = "Explain what a neural network is in one sentence."

response = analyzer.get_response(test_prompt, temperature=0.7)

print("Prompt:", test_prompt)
print("\nResponse:", response['response_text'])
print("\nMetadata:")
print(f"  - Input tokens: {response['input_tokens']}")
print(f"  - Output tokens: {response['output_tokens']}")
#print(f"  - Response length: {response['response_length']} characters")
print(f"  - Stop reason: {response['stop_reason']}")

## 2. Comparing Prompt Engineering Techniques

Now let's compare different prompt engineering approaches on the same question.

In [ ]:
# Define your question
base_question = "What are the three most important factors in training a deep learning model?"

# Define different prompt engineering variants
prompt_variants = {
    "baseline": {
        "prompt": base_question,
        "system": ""
    },
    
    "with_system_prompt": {
        "prompt": base_question,
        "system": "You are an expert deep learning researcher with 10 years of experience."
    },
    
    "chain_of_thought": {
        "prompt": f"{base_question}\n\nLet's think through this step by step:",
        "system": ""
    },
    
    "structured": {
        "prompt": f"{base_question}\n\nProvide your answer in this format:\n1. [Factor]: [Explanation]\n2. [Factor]: [Explanation]\n3. [Factor]: [Explanation]",
        "system": ""
    },
    
    "few_shot": {
        "prompt": f"""Here's an example of explaining key factors:

Q: What are the three most important factors in building a web application?
A: The three most important factors are:
1. Security - Protecting user data and preventing vulnerabilities
2. Performance - Ensuring fast load times and responsive interactions
3. User Experience - Creating an intuitive and accessible interface

Q: {base_question}
A:""",
        "system": ""
    },
    
    "role_playing": {
        "prompt": base_question,
        "system": "You are Yann LeCun, discussing deep learning with a colleague. Be concise but insightful."
    }
}

print("Prompt variants defined:")
for name in prompt_variants.keys():
    print(f"  - {name}")

### Run the comparison (this will take a few minutes)

In [ ]:
# Compare all variants
print("Running comparison... This may take a few minutes.\n")

df_variants = analyzer.compare_prompts(
    base_question=base_question,
    prompt_variants=prompt_variants,
    temperature=0.7,
    num_samples=5  # 5 samples per variant
)

print("\n✓ Comparison complete!")
print(f"\nTotal responses collected: {len(df_variants)}")

### View sample responses

In [ ]:
# Display one sample from each variant
for variant in df_variants['variant'].unique():
    sample = df_variants[df_variants['variant'] == variant].iloc[0]
    print("="*80)
    print(f"Variant: {variant}")
    print("="*80)
    print(f"Response:\n{sample['response_text']}")
    print(f"\nTokens: {sample['output_tokens']} | Length: {sample['response_length']} chars\n")

### Analyze output diversity

In [ ]:
diversity_metrics = analyzer.analyze_output_diversity(df_variants)

# Convert to DataFrame for better display
diversity_df = pd.DataFrame(diversity_metrics).T
diversity_df = diversity_df.round(3)

print("Output Diversity Metrics:\n")
print(diversity_df)

# Highlight key insights
print("\n" + "="*80)
print("KEY INSIGHTS:")
print("="*80)

most_diverse = diversity_df['uniqueness_ratio'].idxmax()
least_diverse = diversity_df['uniqueness_ratio'].idxmin()

print(f"\nMost diverse responses: {most_diverse} (ratio: {diversity_df.loc[most_diverse, 'uniqueness_ratio']:.3f})")
print(f"Least diverse responses: {least_diverse} (ratio: {diversity_df.loc[least_diverse, 'uniqueness_ratio']:.3f})")

longest = diversity_df['avg_response_length'].idxmax()
shortest = diversity_df['avg_response_length'].idxmin()

print(f"\nLongest responses: {longest} ({diversity_df.loc[longest, 'avg_response_length']:.0f} chars avg)")
print(f"Shortest responses: {shortest} ({diversity_df.loc[shortest, 'avg_response_length']:.0f} chars avg)")

### Visualize the comparison

In [ ]:
analyzer.visualize_prompt_comparison(df_variants, save_path="prompt_comparison.png")

## 3. Temperature Analysis

Now let's see how temperature affects the output distribution for a single prompt.

In [ ]:
# Define a prompt for temperature analysis
temp_test_prompt = "Name three creative uses for artificial intelligence in healthcare."

print("Running temperature analysis...\n")
print(f"Prompt: {temp_test_prompt}")
print(f"Temperatures to test: [0.0, 0.3, 0.7, 1.0, 1.5]\n")

df_temperature = analyzer.analyze_temperature_effects(
    prompt=temp_test_prompt,
    system="You are a creative AI assistant.",
    temperatures=[0.0, 0.3, 0.7, 1.0],
    num_samples=10
)

print("\n✓ Temperature analysis complete!")

### View responses at different temperatures

In [ ]:
# Show one example from each temperature
for temp in sorted(df_temperature['temperature'].unique()):
    sample = df_temperature[df_temperature['temperature'] == temp].iloc[0]
    print("="*80)
    print(f"Temperature: {temp}")
    print("="*80)
    print(sample['response_text'])
    print()

### Temperature statistics

In [ ]:
# Calculate statistics by temperature
temp_stats = df_temperature.groupby('temperature').agg({
    'response_length': ['mean', 'std', 'min', 'max'],
    'word_count': ['mean', 'std'],
    'output_tokens': ['mean', 'std']
}).round(2)

print("Statistics by Temperature:\n")
print(temp_stats)

# Calculate uniqueness ratio by temperature
print("\n" + "="*80)
print("Response Uniqueness by Temperature:")
print("="*80)

for temp in sorted(df_temperature['temperature'].unique()):
    responses = df_temperature[df_temperature['temperature'] == temp]['response_text'].tolist()
    unique_ratio = len(set(responses)) / len(responses)
    print(f"Temperature {temp}: {unique_ratio:.2%} unique responses ({len(set(responses))}/{len(responses)})")

### Visualize temperature effects

In [ ]:
analyzer.visualize_temperature_effects(df_temperature, save_path="temperature_effects.png")

## 4. Custom Analysis: Response Patterns

Let's do some custom analysis on common patterns in the responses.

In [ ]:
def analyze_response_patterns(df, text_column='response_text'):
    """
    Analyze common patterns in responses.
    """
    patterns = {
        'starts_with_number': 0,
        'contains_list': 0,
        'contains_bullet': 0,
        'starts_with_capital': 0,
        'contains_colon': 0,
    }
    
    for text in df[text_column]:
        if text[0].isdigit():
            patterns['starts_with_number'] += 1
        if any(text.startswith(f"{i}.") or f"\n{i}." in text for i in range(1, 10)):
            patterns['contains_list'] += 1
        if '•' in text or '- ' in text or '* ' in text:
            patterns['contains_bullet'] += 1
        if text[0].isupper():
            patterns['starts_with_capital'] += 1
        if ':' in text:
            patterns['contains_colon'] += 1
    
    # Convert to percentages
    total = len(df)
    for key in patterns:
        patterns[key] = (patterns[key] / total) * 100
    
    return patterns

# Analyze patterns by variant
print("Response Patterns by Variant:\n")
print("=" * 80)

for variant in df_variants['variant'].unique():
    variant_df = df_variants[df_variants['variant'] == variant]
    patterns = analyze_response_patterns(variant_df)
    
    print(f"\n{variant}:")
    for pattern, percentage in patterns.items():
        print(f"  {pattern}: {percentage:.1f}%")

## 5. Word Frequency Analysis

Analyze which words appear most frequently in different prompt variants.

In [ ]:
from collections import Counter
import re

def get_top_words(df, variant, n=20, min_length=4):
    """
    Get top N words for a specific variant.
    """
    # Get all responses for this variant
    texts = df[df['variant'] == variant]['response_text'].tolist()
    
    # Combine all text
    combined_text = ' '.join(texts).lower()
    
    # Extract words (alphanumeric only)
    words = re.findall(r'\b[a-z]+\b', combined_text)
    
    # Filter by length and remove common stop words
    stop_words = {'the', 'this', 'that', 'with', 'from', 'have', 'they', 'will', 'your', 
                  'more', 'about', 'which', 'their', 'there', 'than', 'them', 'these',
                  'been', 'were', 'when', 'where', 'also', 'can', 'are', 'and', 'for'}
    
    words = [w for w in words if len(w) >= min_length and w not in stop_words]
    
    # Count and return top N
    return Counter(words).most_common(n)

# Analyze top words for each variant
print("Top 15 Words by Variant:\n")
print("=" * 80)

for variant in df_variants['variant'].unique():
    print(f"\n{variant}:")
    top_words = get_top_words(df_variants, variant, n=15)
    
    for word, count in top_words:
        print(f"  {word}: {count}")

## 6. Visualize Word Frequency Comparison

In [ ]:
# Create a comparison of top words across variants
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, variant in enumerate(df_variants['variant'].unique()):
    if idx >= len(axes):
        break
        
    top_words = get_top_words(df_variants, variant, n=10)
    words, counts = zip(*top_words)
    
    axes[idx].barh(words, counts, color=sns.color_palette("husl", 10))
    axes[idx].set_xlabel('Frequency')
    axes[idx].set_title(f'Top Words: {variant}')
    axes[idx].invert_yaxis()

# Hide any unused subplots
for idx in range(len(df_variants['variant'].unique()), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('word_frequency_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Save Results

In [ ]:
# Save all results
analyzer.save_results(df_variants, "prompt_variants_results")
analyzer.save_results(df_temperature, "temperature_analysis_results")

print("Results saved!")
print("\nGenerated files:")
print("  - prompt_variants_results.csv")
print("  - prompt_variants_results.json")
print("  - temperature_analysis_results.csv")
print("  - temperature_analysis_results.json")
print("  - prompt_comparison.png")
print("  - temperature_effects.png")
print("  - word_frequency_comparison.png")

## 8. One-Shot vs Multi-Shot Prompting Analysis

This section analyzes how the number of examples (zero-shot, one-shot, few-shot, multi-shot) affects:
1. **Response Consistency (Stationarity)**: How stable and predictable are the outputs?
2. **Format Adherence**: Does the model follow the format shown in examples?
3. **Response Quality**: How well does it extract structured information?

We'll test this with a product review classification task, extracting sentiment and key features.

In [ ]:
# Define the task: Extract sentiment and key features from a product review
test_review = """I recently purchased the UltraBook Pro laptop and I'm thoroughly impressed. 
The battery life easily lasts 12 hours, the display is crystal clear, and the keyboard is comfortable 
for long typing sessions. However, the price point is quite high and it gets warm during intensive tasks."""

# Define variants with different numbers of examples
shot_variants = {
    "zero_shot": {
        "prompt": f"""Analyze this product review and provide:
1. Overall sentiment (Positive/Negative/Mixed)
2. Key positive features (list)
3. Key negative features (list)

Review: {test_review}""",
        "system": ""
    },
    
    "one_shot": {
        "prompt": f"""Analyze product reviews and extract sentiment and key features.

Example:
Review: "The wireless headphones sound great and are very comfortable. Battery lasts 20 hours. 
But they're expensive and don't fold for storage."

Analysis:
1. Overall sentiment: Mixed
2. Key positive features:
   - Great sound quality
   - Very comfortable
   - 20-hour battery life
3. Key negative features:
   - Expensive
   - Don't fold for storage

Now analyze this review:
Review: {test_review}

Analysis:""",
        "system": ""
    },
    
    "two_shot": {
        "prompt": f"""Analyze product reviews and extract sentiment and key features.

Example 1:
Review: "The wireless headphones sound great and are very comfortable. Battery lasts 20 hours. 
But they're expensive and don't fold for storage."

Analysis:
1. Overall sentiment: Mixed
2. Key positive features:
   - Great sound quality
   - Very comfortable
   - 20-hour battery life
3. Key negative features:
   - Expensive
   - Don't fold for storage

Example 2:
Review: "This smartphone is amazing! Super fast processor, excellent camera, beautiful design. 
Absolutely love it!"

Analysis:
1. Overall sentiment: Positive
2. Key positive features:
   - Super fast processor
   - Excellent camera
   - Beautiful design
3. Key negative features:
   - None mentioned

Now analyze this review:
Review: {test_review}

Analysis:""",
        "system": ""
    },
    
    "multi_shot": {
        "prompt": f"""Analyze product reviews and extract sentiment and key features.

Example 1:
Review: "The wireless headphones sound great and are very comfortable. Battery lasts 20 hours. 
But they're expensive and don't fold for storage."

Analysis:
1. Overall sentiment: Mixed
2. Key positive features:
   - Great sound quality
   - Very comfortable
   - 20-hour battery life
3. Key negative features:
   - Expensive
   - Don't fold for storage

Example 2:
Review: "This smartphone is amazing! Super fast processor, excellent camera, beautiful design. 
Absolutely love it!"

Analysis:
1. Overall sentiment: Positive
2. Key positive features:
   - Super fast processor
   - Excellent camera
   - Beautiful design
3. Key negative features:
   - None mentioned

Example 3:
Review: "Terrible coffee maker. Leaked water everywhere, broke after 2 weeks. Complete waste of money."

Analysis:
1. Overall sentiment: Negative
2. Key positive features:
   - None mentioned
3. Key negative features:
   - Leaks water
   - Poor durability (broke after 2 weeks)
   - Poor value for money

Example 4:
Review: "The fitness tracker works well for basic tracking. Steps and heart rate are accurate. 
However, the app is clunky and syncing is unreliable."

Analysis:
1. Overall sentiment: Mixed
2. Key positive features:
   - Accurate step tracking
   - Accurate heart rate monitoring
3. Key negative features:
   - Clunky app interface
   - Unreliable syncing

Now analyze this review:
Review: {test_review}

Analysis:""",
        "system": ""
    }
}

print("Few-Shot Variants Defined:")
print(f"  - zero_shot: 0 examples")
print(f"  - one_shot: 1 example")
print(f"  - two_shot: 2 examples")
print(f"  - multi_shot: 4 examples")
print(f"\nTest review: {test_review[:100]}...")

### Run the Few-Shot Experiment

We'll run each variant multiple times to measure consistency (stationarity) across responses.

In [ ]:
# Run the experiment - test each variant multiple times
print("Running few-shot analysis... This will take a few minutes.\n")

df_few_shot = analyzer.compare_prompts(
    base_question=test_review,
    prompt_variants=shot_variants,
    temperature=0.7,  # Using temperature 0.7 for some diversity
    num_samples=15  # More samples to better measure stationarity
)

print("\n✓ Few-shot analysis complete!")
print(f"\nTotal responses collected: {len(df_few_shot)}")
print(f"Variants tested: {df_few_shot['variant'].unique().tolist()}")

### Analyze Prompt Stationarity

**Stationarity** refers to how consistent and predictable the model's responses are. 
High stationarity means the model produces similar outputs given the same prompt (desirable for production use).

We'll measure:
1. **Response Variance**: Standard deviation in response length and word count
2. **Format Consistency**: How often responses follow the expected format
3. **Uniqueness Ratio**: Proportion of unique responses (lower = more stationary)
4. **Content Consistency**: Similarity in key terms used across responses

In [ ]:
def analyze_stationarity(df, text_column='response_text'):
    """
    Analyze prompt stationarity - how consistent are the responses?
    
    Returns metrics for each variant:
    - Response length variance (lower = more stationary)
    - Word count variance (lower = more stationary)
    - Uniqueness ratio (lower = more stationary)
    - Format consistency (higher = more stationary)
    - Coefficient of variation for length (normalized measure)
    """
    stationarity_metrics = {}
    
    for variant in df['variant'].unique():
        variant_df = df[df['variant'] == variant]
        responses = variant_df[text_column].tolist()
        
        # Basic stats
        lengths = variant_df['response_length'].values
        word_counts = variant_df['word_count'].values
        
        # Uniqueness
        unique_responses = len(set(responses))
        total_responses = len(responses)
        uniqueness_ratio = unique_responses / total_responses
        
        # Variance metrics
        length_std = np.std(lengths)
        length_mean = np.mean(lengths)
        word_count_std = np.std(word_counts)
        word_count_mean = np.mean(word_counts)
        
        # Coefficient of variation (CV) - normalized measure of dispersion
        # Lower CV = more stationary
        length_cv = (length_std / length_mean) if length_mean > 0 else 0
        word_count_cv = (word_count_std / word_count_mean) if word_count_mean > 0 else 0
        
        # Format consistency - check if responses follow numbered list format
        format_matches = 0
        for response in responses:
            # Check if response contains "1." and "2." and "3."
            if all(f"{i}." in response for i in [1, 2, 3]):
                format_matches += 1
        format_consistency = format_matches / total_responses
        
        # Content consistency - measure overlap in key terms
        # Extract key terms from all responses
        all_terms = []
        for response in responses:
            # Simple term extraction (lowercased words)
            terms = set(response.lower().split())
            all_terms.append(terms)
        
        # Calculate average Jaccard similarity between all pairs
        if len(all_terms) > 1:
            similarities = []
            for i in range(len(all_terms)):
                for j in range(i + 1, len(all_terms)):
                    intersection = len(all_terms[i] & all_terms[j])
                    union = len(all_terms[i] | all_terms[j])
                    similarity = intersection / union if union > 0 else 0
                    similarities.append(similarity)
            avg_content_similarity = np.mean(similarities) if similarities else 0
        else:
            avg_content_similarity = 1.0
        
        stationarity_metrics[variant] = {
            'uniqueness_ratio': uniqueness_ratio,
            'length_std': length_std,
            'length_cv': length_cv,
            'word_count_std': word_count_std,
            'word_count_cv': word_count_cv,
            'format_consistency': format_consistency,
            'content_similarity': avg_content_similarity,
            'num_samples': total_responses,
            # Lower stationarity_score = more stationary
            # Combine multiple metrics (normalized)
            'stationarity_score': (uniqueness_ratio + length_cv + word_count_cv) / 3 - (format_consistency + avg_content_similarity) / 2
        }
    
    return stationarity_metrics

# Analyze stationarity
print("Analyzing prompt stationarity...\n")
stationarity_metrics = analyze_stationarity(df_few_shot)

# Convert to DataFrame for better visualization
stationarity_df = pd.DataFrame(stationarity_metrics).T
stationarity_df = stationarity_df.round(4)

# Sort by number of examples (0, 1, 2, 4)
shot_order = ['zero_shot', 'one_shot', 'two_shot', 'multi_shot']
stationarity_df = stationarity_df.reindex(shot_order)

print("=" * 80)
print("STATIONARITY METRICS BY VARIANT")
print("=" * 80)
print("\nLower values indicate MORE stationary (more consistent) responses")
print("Higher values indicate LESS stationary (more diverse) responses\n")
print(stationarity_df)

# Highlight key findings
print("\n" + "=" * 80)
print("KEY FINDINGS:")
print("=" * 80)

most_stationary = stationarity_df['stationarity_score'].idxmin()
least_stationary = stationarity_df['stationarity_score'].idxmax()

print(f"\nMost Stationary (most consistent): {most_stationary}")
print(f"  - Stationarity Score: {stationarity_df.loc[most_stationary, 'stationarity_score']:.4f}")
print(f"  - Uniqueness Ratio: {stationarity_df.loc[most_stationary, 'uniqueness_ratio']:.2%}")
print(f"  - Format Consistency: {stationarity_df.loc[most_stationary, 'format_consistency']:.2%}")
print(f"  - Content Similarity: {stationarity_df.loc[most_stationary, 'content_similarity']:.2%}")

print(f"\nLeast Stationary (most diverse): {least_stationary}")
print(f"  - Stationarity Score: {stationarity_df.loc[least_stationary, 'stationarity_score']:.4f}")
print(f"  - Uniqueness Ratio: {stationarity_df.loc[least_stationary, 'uniqueness_ratio']:.2%}")
print(f"  - Format Consistency: {stationarity_df.loc[least_stationary, 'format_consistency']:.2%}")
print(f"  - Content Similarity: {stationarity_df.loc[least_stationary, 'content_similarity']:.2%}")

### Visualize Sample Responses

Let's look at a few sample responses from each variant to see the differences.

In [ ]:
# Display 2 sample responses from each variant
for variant in shot_order:
    variant_responses = df_few_shot[df_few_shot['variant'] == variant]['response_text'].tolist()[:2]
    
    print("=" * 80)
    print(f"VARIANT: {variant.upper()} ({variant.split('_')[0]} examples)")
    print("=" * 80)
    
    for idx, response in enumerate(variant_responses, 1):
        print(f"\nSample {idx}:")
        print("-" * 80)
        print(response)
        print()

### Visualize Stationarity Metrics

In [ ]:
# Create comprehensive visualizations for stationarity analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Stationarity Score (lower = more stationary)
ax = axes[0, 0]
colors = sns.color_palette("RdYlGn_r", len(shot_order))
bars = ax.bar(shot_order, stationarity_df['stationarity_score'], color=colors)
ax.set_xlabel('Variant (Number of Examples)')
ax.set_ylabel('Stationarity Score (lower = more consistent)')
ax.set_title('Overall Stationarity Score by Variant')
ax.set_xticklabels(['0-shot', '1-shot', '2-shot', '4-shot'])
ax.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax.grid(axis='y', alpha=0.3)

# 2. Uniqueness Ratio (lower = more stationary)
ax = axes[0, 1]
bars = ax.bar(shot_order, stationarity_df['uniqueness_ratio'], color=sns.color_palette("Reds_r", len(shot_order)))
ax.set_xlabel('Variant (Number of Examples)')
ax.set_ylabel('Uniqueness Ratio (lower = more consistent)')
ax.set_title('Response Uniqueness by Variant')
ax.set_xticklabels(['0-shot', '1-shot', '2-shot', '4-shot'])
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

# 3. Format Consistency (higher = more stationary)
ax = axes[0, 2]
bars = ax.bar(shot_order, stationarity_df['format_consistency'], color=sns.color_palette("Greens", len(shot_order)))
ax.set_xlabel('Variant (Number of Examples)')
ax.set_ylabel('Format Consistency (higher = better)')
ax.set_title('Format Adherence by Variant')
ax.set_xticklabels(['0-shot', '1-shot', '2-shot', '4-shot'])
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

# 4. Content Similarity (higher = more stationary)
ax = axes[1, 0]
bars = ax.bar(shot_order, stationarity_df['content_similarity'], color=sns.color_palette("Blues", len(shot_order)))
ax.set_xlabel('Variant (Number of Examples)')
ax.set_ylabel('Content Similarity (higher = more consistent)')
ax.set_title('Average Content Similarity by Variant')
ax.set_xticklabels(['0-shot', '1-shot', '2-shot', '4-shot'])
ax.set_ylim(0, 1.1)
ax.grid(axis='y', alpha=0.3)

# 5. Response Length Coefficient of Variation (lower = more stationary)
ax = axes[1, 1]
bars = ax.bar(shot_order, stationarity_df['length_cv'], color=sns.color_palette("Oranges_r", len(shot_order)))
ax.set_xlabel('Variant (Number of Examples)')
ax.set_ylabel('Length CV (lower = more consistent)')
ax.set_title('Response Length Variability')
ax.set_xticklabels(['0-shot', '1-shot', '2-shot', '4-shot'])
ax.grid(axis='y', alpha=0.3)

# 6. Response Length Distribution by Variant
ax = axes[1, 2]
for variant in shot_order:
    variant_data = df_few_shot[df_few_shot['variant'] == variant]['response_length']
    ax.boxplot([variant_data], positions=[shot_order.index(variant)], widths=0.6, patch_artist=True,
                boxprops=dict(facecolor=colors[shot_order.index(variant)], alpha=0.7))

ax.set_xlabel('Variant (Number of Examples)')
ax.set_ylabel('Response Length (characters)')
ax.set_title('Response Length Distribution')
ax.set_xticks(range(len(shot_order)))
ax.set_xticklabels(['0-shot', '1-shot', '2-shot', '4-shot'])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('few_shot_stationarity_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved as 'few_shot_stationarity_analysis.png'")

### Save Few-Shot Results

In [ ]:
# Save the few-shot results
analyzer.save_results(df_few_shot, "few_shot_results")

# Also save the stationarity metrics
stationarity_df.to_csv("stationarity_metrics.csv")
stationarity_df.to_json("stationarity_metrics.json", orient='index', indent=2)

print("Results saved!")
print("\nGenerated files:")
print("  - few_shot_results.csv")
print("  - few_shot_results.json")
print("  - stationarity_metrics.csv")
print("  - stationarity_metrics.json")
print("  - few_shot_stationarity_analysis.png")

### Key Insights: Impact of Examples on Prompt Stationarity

**What we learned:**

1. **More examples generally increase stationarity** - Few-shot prompts (especially multi-shot with 4 examples) tend to produce more consistent, predictable outputs.

2. **Format consistency improves with examples** - When you show the model the desired format through examples, it's more likely to follow that format consistently.

3. **Content similarity increases with examples** - More examples help the model converge on similar content and vocabulary across multiple runs.

4. **Trade-off: Consistency vs. Creativity** 
   - Zero-shot: More diverse, creative responses but less predictable
   - Few-shot: More consistent, reliable responses but potentially less creative

5. **Practical implications:**
   - For **production systems** where reliability matters: Use few-shot or multi-shot prompts
   - For **creative applications** where diversity is valuable: Zero-shot or one-shot may be better
   - For **structured extraction tasks**: Multi-shot prompts significantly improve format adherence

6. **Diminishing returns** - Going from 2 to 4 examples may show smaller improvements than going from 0 to 1 example. Test to find the optimal number for your use case.

## 9. Your Custom Experiments

Use this section to run your own experiments!

In [ ]:
# Define your own question and variants here

my_question = "Your question here"

my_variants = {
    "variant_1": {
        "prompt": my_question,
        "system": ""
    },
    # Add more variants...
}

# Run your experiment
# my_results = analyzer.compare_prompts(my_question, my_variants, temperature=0.7, num_samples=5)
# analyzer.visualize_prompt_comparison(my_results)

## Summary

This notebook demonstrated:

1. **Prompt Engineering Techniques**: How different prompting strategies (zero-shot, few-shot, chain-of-thought, etc.) affect outputs
2. **Temperature Effects**: How temperature influences output diversity and creativity
3. **Output Analysis**: Measuring response length, diversity, and patterns
4. **Word Frequency**: Understanding vocabulary usage across different prompts
5. **Few-Shot Learning Impact on Stationarity**: How the number of examples affects response consistency and predictability

Key findings you might observe:
- Lower temperatures (0.0-0.3) produce more consistent, deterministic outputs
- Higher temperatures (1.0-1.5) increase diversity but may reduce coherence
- Structured prompts tend to produce more consistent formatting
- System prompts can significantly influence tone and style
- Few-shot examples guide the model toward specific response patterns
- **More examples increase stationarity**: Multi-shot prompts (4 examples) produce the most consistent, predictable outputs
- **Format adherence improves with examples**: Few-shot learning significantly improves structured output compliance
- **Trade-off between consistency and creativity**: Zero-shot prompts are more creative but less reliable; multi-shot prompts are more consistent but potentially less diverse